In [ ]:
Criando o banco de dados

In [1]:
spark.sql("CREATE DATABASE IF NOT EXISTS earth")

DataFrame[]

Dropando tabelas que serão criadas caso elas existam

In [5]:
%%sql
DROP TABLE IF EXISTS earth.climate_change_partitioned

++
||
++
++

In [6]:
%%sql
DROP TABLE IF EXISTS earth.climate_change

++
||
++
++

Criando tabelas no padrão Iceberg, uma normal e uma particionada

In [8]:
df = spark.read.parquet("/home/iceberg/data/climate_change.parquet")
df.writeTo("icelake.earth.climate_change_partitioned").partitionedBy("Country").createOrReplace()

In [9]:
temp_df = spark.read.parquet("/home/iceberg/data/climate_change.parquet")
temp_df.writeTo("earth.climate_change").createOrReplace()

Checando o país com mais resultados de medições

In [15]:
spark.sql("SELECT Country, COUNT(1) FROM earth.climate_change GROUP BY 1 ORDER BY 2 DESC LIMIT 5").show()

+-------------+--------+
|      Country|count(1)|
+-------------+--------+
|        India| 1014906|
|        China|  827802|
|United States|  687289|
|       Brazil|  475580|
|       Russia|  461234|
+-------------+--------+



Demonstração do plano de execução do ambiente particionado e normal

In [27]:
nao_part = spark.sql("SELECT * FROM earth.climate_change WHERE Country = 'India' ") #Tabela Não Particionada
nao_part.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('Country = India)
   +- 'UnresolvedRelation [earth, climate_change], [], false

== Analyzed Logical Plan ==
dt: string, AverageTemperature: double, AverageTemperatureUncertainty: double, City: string, Country: string, Latitude: string, Longitude: string
Project [dt#332, AverageTemperature#333, AverageTemperatureUncertainty#334, City#335, Country#336, Latitude#337, Longitude#338]
+- Filter (Country#336 = India)
   +- SubqueryAlias icelake.earth.climate_change
      +- RelationV2[dt#332, AverageTemperature#333, AverageTemperatureUncertainty#334, City#335, Country#336, Latitude#337, Longitude#338] icelake.earth.climate_change icelake.earth.climate_change

== Optimized Logical Plan ==
Filter (isnotnull(Country#336) AND (Country#336 = India))
+- RelationV2[dt#332, AverageTemperature#333, AverageTemperatureUncertainty#334, City#335, Country#336, Latitude#337, Longitude#338] icelake.earth.climate_change

== Physical Plan ==
*(1) Filter (isnot

In [30]:
part = spark.sql("SELECT * FROM earth.climate_change_partitioned WHERE Country = 'India'") #Tabela Particionada
part.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('Country = India)
   +- 'UnresolvedRelation [earth, climate_change_partitioned], [], false

== Analyzed Logical Plan ==
dt: string, AverageTemperature: double, AverageTemperatureUncertainty: double, City: string, Country: string, Latitude: string, Longitude: string
Project [dt#384, AverageTemperature#385, AverageTemperatureUncertainty#386, City#387, Country#388, Latitude#389, Longitude#390]
+- Filter (Country#388 = India)
   +- SubqueryAlias icelake.earth.climate_change_partitioned
      +- RelationV2[dt#384, AverageTemperature#385, AverageTemperatureUncertainty#386, City#387, Country#388, Latitude#389, Longitude#390] icelake.earth.climate_change_partitioned icelake.earth.climate_change_partitioned

== Optimized Logical Plan ==
RelationV2[dt#384, AverageTemperature#385, AverageTemperatureUncertainty#386, City#387, Country#388, Latitude#389, Longitude#390] icelake.earth.climate_change_partitioned

== Physical Plan ==
*(1) ColumnarToRow
